# There are 8000 records which are less than 0
# there are 40 items = 0


In [113]:
import sqlite3
import datetime as dt

In [35]:
file = open("C:\\DataSets\\OnlineRetail2Sample.csv")
data1 = file.readlines()
data = data1[1:]

In [36]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")

Opened database successfully


In [ ]:
conn.execute('''CREATE TABLE RFMDM (
    InvoiceNo   CHAR (20),
    StockCode   CHAR (20),
    Quantity    INTEGER,
    DoP         DATE,
    UnitPrice   FLOAT,
    CustomerID  CHAR (30));''')
print ("Table created successfully")

#conn.close()

In [37]:
for row in data:
    #iData = data[row].split(sep = ",")
    iData = row.split(sep = ",")
    #print(iData)
    InvoiceNo = iData[0]
    StockCode = iData[1]
    Quantity = iData[2]
    DoP = iData[3]
    UnitPrice = iData[4]
    CustomerID = iData[5]

    #print(InvoiceNo, StockCode, Quantity, DoP, UnitPrice, CustomerID)

    #sql = 'INSERT INTO RFMDM (InvoiceNo,StockCode,Quantity,DoP,UnitPrice,CustomerID) \
    #      VALUES (' + 536365 , 71053, 6, \'12/1/2010\', 3.39, 17850)"

    sql = "INSERT INTO RFMDM (InvoiceNo,StockCode,Quantity,DoP,UnitPrice,CustomerID) \
          VALUES (" + "\'" + InvoiceNo + "\',\'"+ StockCode + "\',"+ Quantity  + ','+ "\'" + DoP + "\'" +','+ UnitPrice +',' + CustomerID + " )"
    #print(sql)
    conn.execute(sql)
conn.commit()    
conn.close()
print("Commit successful")

Commit successful


In [38]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cur = conn.execute("select * from RFMDM limit 5")
data = cur.fetchall()
conn.close()
print(data)

Opened database successfully
[('536365', '85123A', 6, '12/1/2010', 2.55, '17850'), ('536365', '71053', 6, '12/1/2010', 3.39, '17850'), ('536365', '84406B', 8, '12/1/2010', 2.75, '17850'), ('536365', '84029G', 6, '12/1/2010', 3.39, '17850'), ('536365', '84029E', 6, '12/1/2010', 3.39, '17850')]


#Run delete query in the table rfmdm
select count(*) from RFMDM
-- 406829

select count(*) from rfmdm where StockCode in ('POST', 'D', 'BANK CHARGES', 'C2', 'CRUK', 'DOT', 'M', 'PADS')

--1920

delete from rfmdm where StockCode in ('POST', 'D', 'BANK CHARGES', 'C2', 'CRUK', 'DOT', 'M', 'PADS')
select count(*) from RFMDM where Quantity<0

--8359 (2%)

delete from rfmdm where quantity <0

In [103]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")

#conn.execute('''drop table rfmseg''')
conn.execute('''CREATE TABLE RFMSeg (
    CustomerID  CHAR (20),
    RDate   DATE,
    Frequency    INTEGER,
    Product INTEGER,
    TDate         DATE,
    Monetary   FLOAT)''')
print ("Table created successfully")

conn.close() 

Opened database successfully
Table created successfully


In [39]:
import pandas as pd

In [104]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
rfmdm = pd.read_sql("select * from RFMDM",conn)
print ("records added to dataframe successfully")
conn.close()

Opened database successfully
records added to dataframe successfully


In [105]:
rfmdm.head()

,InvoiceNo,StockCode,Quantity,DoP,UnitPrice,CustomerID
0,536365,85123A,6,12/1/2010,2.55,17850
1,536365,71053,6,12/1/2010,3.39,17850
2,536365,84406B,8,12/1/2010,2.75,17850
3,536365,84029G,6,12/1/2010,3.39,17850
4,536365,84029E,6,12/1/2010,3.39,17850


In [106]:
rfmdm.dtypes

InvoiceNo      object
StockCode      object
Quantity        int64
DoP            object
UnitPrice     float64
CustomerID     object
dtype: object

In [107]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cur = conn.execute("select distinct(DoP) from RFMDM limit 5")
data = cur.fetchall()
conn.close()
print(data)

Opened database successfully
[('12/1/2010',), ('12/2/2010',), ('12/3/2010',), ('12/5/2010',), ('12/6/2010',)]


In [108]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cur = conn.execute("select count(*) from RFMDM")
data = cur.fetchall()
conn.close()
print(data)

Opened database successfully
[(406829,)]


In [109]:
rfmdm['DoP'] = pd.to_datetime(rfmdm['DoP'])
print("Converted successfully")

Converted successfully


In [110]:
rfmdm.dtypes

InvoiceNo             object
StockCode             object
Quantity               int64
DoP           datetime64[ns]
UnitPrice            float64
CustomerID            object
dtype: object

In [ ]:
#Fix datetime format issue

In [111]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
rfmdm.to_sql('rfmdm_final',conn, if_exists = 'replace')
#rfmdm = pd.read_sql("select * from RFMDM",conn)
print ("new table got created successfully")
conn.close()

Opened database successfully
new table got created successfully


In [112]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cur = conn.execute("select distinct(DoP) from RFMDM_final limit 5")
data = cur.fetchall()
conn.close()
print(data)

Opened database successfully
[('2010-12-01 00:00:00',), ('2010-12-02 00:00:00',), ('2010-12-03 00:00:00',), ('2010-12-05 00:00:00',), ('2010-12-06 00:00:00',)]


In [ ]:
insert data into rfmseg table using sql
    INSERT INTO RFMSeg
       SELECT CustomerID, max(DoP), count(distinct(InvoiceNo)), count(distinct(StockCode)), min(DoP), sum(quantity*unitprice)
       FROM RFMDM_final
       group by customerid

In [114]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
cur = conn.execute("select * from RFMSEG limit 5")
data = cur.fetchall()
conn.close()
print(data)

Opened database successfully
[('12346', '2011-01-18 00:00:00', 2, 1, '2011-01-18 00:00:00', 0.0), ('12347', '2011-12-07 00:00:00', 7, 103, '2010-12-07 00:00:00', 4309.999999999997), ('12348', '2011-09-25 00:00:00', 4, 22, '2010-12-16 00:00:00', 1797.24), ('12349', '2011-11-21 00:00:00', 1, 73, '2011-11-21 00:00:00', 1757.55), ('12350', '2011-02-02 00:00:00', 1, 17, '2011-02-02 00:00:00', 334.40000000000003)]


In [85]:
print(max(rfmdm['DoP']))

2011-12-09 00:00:00


In [49]:
print(min(rfmdm['DoP']))

2010-12-01 00:00:00


In [78]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")

sql = ''' alter table rfmseg \
            add column Recency INTEGER \
        '''
print(sql)
conn.execute(sql)
conn.commit
print ("Column added successfully")

conn.close()

Opened database successfully
 alter table rfmseg             add column Recency INTEGER         
Column added successfully


In [79]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")

sql = ''' alter table rfmseg \
            add column Tenure INTEGER \
        '''
print(sql)
conn.execute(sql)
conn.commit
print ("Column added successfully")

conn.close()

Opened database successfully
 alter table rfmseg             add column Tenure INTEGER         
Column added successfully


In [92]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
rfmseg = pd.read_sql("select * from RFMSEG",conn)
print ("records added to dataframe rfmseg successfully")
conn.close()

Opened database successfully
records added to dataframe rfmseg successfully


In [116]:
rfmseg.head()
#print(rfmseg['Recency'])

,CustomerID,RDate,Frequency,Product,TDate,Monetary,Recency,Tenure
0,12346,2011-01-18 00:00:00,2,1,2011-01-18 00:00:00,0.00,0,0
1,12347,2011-12-07 00:00:00,7,103,2010-12-07 00:00:00,4310.00,0,1
2,12348,2011-09-25 00:00:00,4,22,2010-12-16 00:00:00,1797.24,0,1
3,12349,2011-11-21 00:00:00,1,73,2011-11-21 00:00:00,1757.55,0,0
4,12350,2011-02-02 00:00:00,1,17,2011-02-02 00:00:00,334.40,0,0


In [130]:

NOW = dt.datetime(2011,12,15)
print(NOW)

2011-12-15 00:00:00


In [131]:
rfmseg['NOW'] = NOW
rfmseg.head()

,CustomerID,RDate,Frequency,Product,TDate,Monetary,Recency,Tenure,NOW
0,12346,2011-01-18,2,1,2011-01-18,0.00,1427,1427,2011-12-15
1,12347,2011-12-07,7,103,2010-12-07,4310.00,1104,1469,2011-12-15
2,12348,2011-09-25,4,22,2010-12-16,1797.24,1177,1460,2011-12-15
3,12349,2011-11-21,1,73,2011-11-21,1757.55,1120,1120,2011-12-15
4,12350,2011-02-02,1,17,2011-02-02,334.40,1412,1412,2011-12-15


In [132]:
rfmseg['NOW'] = pd.to_datetime(rfmseg['NOW'])
rfmseg['TDate'] = pd.to_datetime(rfmseg['TDate'])
rfmseg['RDate'] = pd.to_datetime(rfmseg['RDate'])
rfmseg.head()

,CustomerID,RDate,Frequency,Product,TDate,Monetary,Recency,Tenure,NOW
0,12346,2011-01-18,2,1,2011-01-18,0.00,1427,1427,2011-12-15
1,12347,2011-12-07,7,103,2010-12-07,4310.00,1104,1469,2011-12-15
2,12348,2011-09-25,4,22,2010-12-16,1797.24,1177,1460,2011-12-15
3,12349,2011-11-21,1,73,2011-11-21,1757.55,1120,1120,2011-12-15
4,12350,2011-02-02,1,17,2011-02-02,334.40,1412,1412,2011-12-15


In [133]:
rfmseg.dtypes

CustomerID            object
RDate         datetime64[ns]
Frequency              int64
Product                int64
TDate         datetime64[ns]
Monetary             float64
Recency                int64
Tenure                 int64
NOW           datetime64[ns]
dtype: object

In [134]:
rfmseg['Recency'] = (rfmseg['NOW'] - rfmseg['RDate']).dt.days
rfmseg['Tenure'] = (rfmseg['NOW'] - rfmseg['TDate']).dt.days
rfmseg.head()

,CustomerID,RDate,Frequency,Product,TDate,Monetary,Recency,Tenure,NOW
0,12346,2011-01-18,2,1,2011-01-18,0.00,331,331,2011-12-15
1,12347,2011-12-07,7,103,2010-12-07,4310.00,8,373,2011-12-15
2,12348,2011-09-25,4,22,2010-12-16,1797.24,81,364,2011-12-15
3,12349,2011-11-21,1,73,2011-11-21,1757.55,24,24,2011-12-15
4,12350,2011-02-02,1,17,2011-02-02,334.40,316,316,2011-12-15


In [135]:
conn = sqlite3.connect('C:\\DataSets\\RFMDB.db')
print ("Opened database successfully")
rfmseg.to_sql('rfmseg_final',conn, if_exists = 'replace')
#rfmdm = pd.read_sql("select * from RFMDM",conn)
print ("new table got created successfully")
conn.close()

Opened database successfully
new table got created successfully


In [84]:
#print(max(rfmseg['RDate']))
#rfmseg['MAXRDATE'] = max(rfmseg['RDate'])
#rfmseg['reportdate'] = pd.to_datetime(rfmseg['MAXRDATE']).apply(pd.DateOffset(1))
rfmseg.head()

12/23/2010


,CustomerID,RDate,Frequency,Product,TDate,Monetary,Recency,Tenure,MAXRDATE,reportdate
0,12346,01/18/2011,1,1,01/18/2011,77183.60,None,None,12/23/2010,2010-12-26
1,12347,12/07/2011,7,103,12/07/2010,4310.00,None,None,12/23/2010,2010-12-26
2,12348,09/25/2011,4,21,12/16/2010,1437.24,None,None,12/23/2010,2010-12-26
3,12349,11/21/2011,1,72,11/21/2011,1457.55,None,None,12/23/2010,2010-12-26
4,12350,02/02/2011,1,16,02/02/2011,294.40,None,None,12/23/2010,2010-12-26


In [24]:
rfmseg.describe()

,Frequency,Product,Monetary
count,4335.000000,4335.000000,4335.000000
mean,4.245675,61.413149,2021.007301
std,7.636337,85.304170,8906.526607
min,1.000000,1.000000,0.000000
25%,1.000000,16.000000,305.410000
50%,2.000000,35.000000,668.110000
75%,5.000000,77.000000,1631.475000
max,206.000000,1785.000000,279138.020000


In [ ]:
rfmseg.shape

In [ ]:
import matplotlib.pyplot as plt
rfmseg.hist(figsize = (10,20))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
rfmseg.plot.box()
plt.show()

In [ ]:
plt.boxplot(rfmseg["Frequency"])
plt.show()

In [ ]:
plt.boxplot(rfmseg["Monetary"])
plt.show()

In [ ]:
plt.boxplot(rfmseg["Product"])
plt.show()

In [ ]:
plt.boxplot(rfmseg["Recency"])
plt.show()

In [ ]:
plt.boxplot(rfmseg["Tenure"])
plt.show()

In [ ]:
import seaborn as sns
sns.pairplot(rfmseg, hue = None, size=3)
plt.show()